# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Learning Objectives

At the end of the experiment, you will be able to:


*  Classify the Fashion MNIST dataset using a teacher network  
*  Understand how the student network uses the outputs of the teacher network
*  Establish that student - teacher network improves classification accuracy 

In [ ]:
#@title Experiment Explanation Video
from IPython.display import HTML

HTML("""<video width="800" height="300" controls>
  <source src="https://cdn.talentsprint.com/aiml/AIML_BATCH_HYD_7/March31/student_teacher_network.mp4" type="video/mp4">
</video>
""")

## Dataset

### History

The original MNIST dataset contains handwritten digits. People from AI/ML or Data Science community love this dataset. They use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset they would try on. As per popular belief, If the algorithm doesn’t work on MNIST, it won’t work at all. Well, if algorithm works on MNIST, it may still fail on other datasets.


As per the original [paper](https://arxiv.org/abs/1708.07747) describing about Fashion-MNIST, It is a dataset recomposed from the product pictures of Zalando’s websites. Fashion-MNIST is intended to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms, as it shares the same image size, data format and the structure of training and testing splits.

There are some good reasons for the challenges faced by MNIST dataset:

* MNIST is too easy - Neural networks can achieve 99.7% on MNIST easily, and similarly, even classic ML algorithms can achieve 97%. 

* MNIST is overused - Almost everyone who has experience with deep learning has come across MNIST at least once.

* MNIST cannot represent modern CV task





### Description

The dataset choosen for this experiment is Fashion-MNIST. The dataset is made up of 28x28 grayscale images of 70,000 fashion products from 10 categories, with 7,000 images per category. The training set has 60,000 images and the test set has 10,000 images. 

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255.

**Labels / Classes**

0 - T-shirt/top

1 - Trouser

2 - Pullover

3 - Dress

4 - Coat

5 - Sandal

6 - Shirt

7 - Sneaker

8 - Bag

9 - Ankle boot

## AI / ML Technique

### Student - Teacher Network 

In this experiment, we train our Student - Teacher Network to classify images in the dataset. There are several ways to perform image classification. However, the performance of image classification can often be significantly improved by combining multiple models together which are called ensemble methods. Though beneficial, ensemble methods can be computationally expensive.

Therefore, an alternative approach, appropriate for deep learning schemes, is to adopt student-teacher training. 

In this training approach, there is a huge neural network known as the teacher network which performs classification tasks. The standard approach is to train the student model (this model/network is desirably smaller) on the posterior outputs of the teacher network.

You can look at the student - teacher network sample below:

<img src = "https://3.bp.blogspot.com/-eD3Mc4FLsvA/WvN6BweGY_I/AAAAAAAACuU/OZqGR1UUvL05Ctr1b8JD3SaKlCNCZVMdACLcBGAs/s1600/f2.png" width="600" height="500">




### Importing required packages

In [ ]:
import torch 
import torch.nn as nn
from torchvision import datasets, transforms
import numpy as np

### Hyperparameters

In [ ]:
num_epochs = 10
batch_size = 100
learning_rate = 0.001

### Initializing CUDA



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Downloading Fashion MNIST data

In [ ]:
train_dataset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/',
                           train=False, 
                           transform=transforms.ToTensor())

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



### Dataloader

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

### Defining the Teacher Network

A comparitively bigger and deeper network as compared to the student network defined later.

In [ ]:
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(7*7*32, 300)
        self.fc2 = nn.Linear(300, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    

### Defining the student network

A comparatively smaller and shallower network than the teacher.

In [ ]:
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(14*14*16, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

<b>The below function is called to reinitialize the weights of the network and define the required loss criterion and the optimizer.</b> 

In [ ]:
def reset_model(is_teacher = True):
    if is_teacher == True:
        net = Teacher()
    else:
        net = Student()
        
    net = net.to(device)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    return net, criterion, optimizer

### Training the teacher network

The first step is to train the teacher network to become an expert. We move ahead with regular training procedure using the cross entropy loss and the Adam optimizer.

In [ ]:
teacher, criterion, optimizer = reset_model()

In [ ]:
# Train the Model

def training(net, reset = True):
    if reset == True:
        net, criterion, optimizer = reset_model()
    else:
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    net.train()
    for epoch in range(num_epochs):
        total_loss = 0
        accuracy = []
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
            temp_labels = labels
        

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == temp_labels).sum().item()
            accuracy.append(correct/float(batch_size))

        print('Epoch: %d, Loss: %.4f, Accuracy: %.4f' %(epoch+1,total_loss, (sum(accuracy)/float(len(accuracy)))))
    
    return net

### Testing the teacher network

In [ ]:
# Test the Model
def testing(net):
    net.eval() 
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the network on the 10000 test images: %.2f %%' % (100.0 * correct / total))

In [ ]:
reset = True
teacher = training(teacher, reset)
testing(teacher)

Epoch: 1, Loss: 248.6643, Accuracy: 0.8504
Epoch: 2, Loss: 169.4783, Accuracy: 0.8985
Epoch: 3, Loss: 150.7932, Accuracy: 0.9089
Epoch: 4, Loss: 137.2542, Accuracy: 0.9183
Epoch: 5, Loss: 127.2232, Accuracy: 0.9233
Epoch: 6, Loss: 118.4046, Accuracy: 0.9284
Epoch: 7, Loss: 112.6422, Accuracy: 0.9311
Epoch: 8, Loss: 108.2238, Accuracy: 0.9333
Epoch: 9, Loss: 101.3710, Accuracy: 0.9385
Epoch: 10, Loss: 95.7236, Accuracy: 0.9417
Test Accuracy of the network on the 10000 test images: 90.52 %


### Parameters for Student Network

Here, we define a few more parameters of the student network. In the student network, we will train with the soft targets as well the hard targets. The soft targets will be calculated by the following equation:

$$
f(z_{i}) = \frac{\exp(z_{i})}{\sum_{j}\exp(z_{j})}
$$

This results in softening out the outputs of the teacher and this can be used as hints for the student network.

The loss doesn't need to get backpropagated across the teacher network and therefore we make the corresponding modification.

Also, for training with the soft labels, we use mean square error loss since using a Cross Entropy loss for soft labels makes no sense.

In [ ]:
temperature = 1.5
for p in teacher.parameters():
    p.requires_grad= False

student, criterion, optimizer = reset_model(is_teacher = False)
alpha = 0.6

mse_criterion = nn.MSELoss()
softmax = nn.Softmax()

print(student)

Student(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=10, bias=True)
)


### Training and testing the student network

In [ ]:
# Train the Model

for epoch in range(num_epochs):
    total_loss = 0
    accuracy = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        temp_labels = labels
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        
        student_outputs = student(images)
        
        hard_outputs = teacher(images)
        soft_outputs = hard_outputs/ temperature
        soft_outputs = softmax(soft_outputs)
        
        hard_loss = criterion(student_outputs, labels)
        soft_loss = mse_criterion(student_outputs, soft_outputs)
        loss = alpha*hard_loss + (1-alpha)*soft_loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(student_outputs.data, 1)
        correct = (predicted == temp_labels).sum().item()
        accuracy.append(correct/float(batch_size))
    
    print('Epoch: %d, Loss: %.4f, Accuracy: %.4f' %(epoch+1,total_loss, (sum(accuracy)/float(len(accuracy)))))

Epoch: 1, Loss: 400.4683, Accuracy: 0.8503
Epoch: 2, Loss: 366.5124, Accuracy: 0.8854
Epoch: 3, Loss: 357.3928, Accuracy: 0.8938
Epoch: 4, Loss: 351.7768, Accuracy: 0.8999
Epoch: 5, Loss: 348.9338, Accuracy: 0.9026
Epoch: 6, Loss: 346.3709, Accuracy: 0.9055
Epoch: 7, Loss: 344.3965, Accuracy: 0.9072
Epoch: 8, Loss: 342.7915, Accuracy: 0.9090
Epoch: 9, Loss: 341.4703, Accuracy: 0.9098
Epoch: 10, Loss: 340.4061, Accuracy: 0.9112


In [ ]:
testing(student)

Test Accuracy of the network on the 10000 test images: 89.40 %


### References

1. https://arxiv.org/abs/1412.6550
2. https://www.cs.toronto.edu/~hinton/absps/distillation.pdf